In [12]:
# %pip install accelerate # charset-normalizer  # pandas python-dotenv transformers
%pip install --upgrade accelerate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from huggingface_hub import login
from dotenv import load_dotenv
from torch.utils.data import Dataset

import pandas as pd

import torch
import os

c:\Compilers\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables from .env file
load_dotenv()

token = os.getenv("HUGGINGFACE_API_TOKEN")

login(token)

In [3]:
model_id = "meta-llama/Llama-3.2-1B"

pipe = pipeline(
    "text-generation", 
    model=model_id, 
    torch_dtype=torch.bfloat16,
    max_length=50
)

# response = pipe("What is the most common eaten foodstuff in USA?", return_full_text=False, truncation=True)
response = pipe("What is the most common eaten foodstuff in USA?", truncation=True)
response

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'What is the most common eaten foodstuff in USA? \xa0That’s a question I’ve been asked many times.\xa0 I don’t think it’s a question I’ve ever really been asked before.\xa0 I’m not sure why, but'}]

In [ ]:
from charset_normalizer import detect

# Read a sample of the file
with open('data/Foodex1.csv', 'rb') as file:
    raw_data = file.read()

# Detect encoding
result = detect(raw_data)
print(f"Detected encoding: {result['encoding']}")

encoding = result['encoding']

Detected encoding: None


In [ ]:
# FAOSTAT script
def get_most_eaten_food_in_faostat_dataset():
    faostat = pd.read_csv("data/FAOSTAT_food_consumption.csv")
    # print(faostat['Item'].unique())
    # faostat[faostat['Area'] == 'Afghanistan'].head(50)

    faostat_filtered_units = faostat[faostat['Unit'] == '1000 t']

    faostat_filtered_units = faostat_filtered_units[faostat_filtered_units['Value'] > 0]

    unique_countries = faostat_filtered_units['Area'].unique()

    for country in unique_countries:
        country_data = faostat_filtered_units[faostat_filtered_units['Area'] == country]

        most_eaten_food = country_data[country_data['Value'] == country_data['Value'].max()]['Item'].values[0]

        print(f"In {country} the most eaten food is {most_eaten_food}")

# Dishes script
def get_most_eaten_food_in_dishes_dataset():
    sentences = []

    dish = pd.read_csv("data/dishes.csv")

    unique_countries = dish['countries'].unique()

    for country in unique_countries:
        country_data = dish[dish['countries'] == country]

        country_regions = country_data['regions'].unique()

        for region in country_regions:
            region_data = country_data[country_data['regions'] == region]

            if len(region_data['english_name'].values) == 0:
                continue
            
            sentences.append(f"In {country}, {region} the most eaten food is {region_data['english_name'].values[0]}")
            # print(f"In {country}, {region} the most eaten food is {region_data['english_name'].values[0]}")

    return sentences

In [7]:
# Custom Dataset class for text generation
class TextGenerationDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize the text with padding and truncation
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        # Labels are the same as input_ids for causal language modeling
        encoding["labels"] = encoding["input_ids"]
        return {key: val.squeeze(0) for key, val in encoding.items()}

In [ ]:
# Load the pretrained text-generation model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Ensure padding tokens are set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Loading dataset
texts = get_most_eaten_food_in_dishes_dataset()

# Prepare the dataset
train_dataset = TextGenerationDataset(texts, tokenizer)

# Training arguments
training_args = TrainingArguments(
    output_dir="trainer",               # Output directory
    learning_rate=5e-5,                 # Learning rate
    per_device_train_batch_size=2,      # Batch size
    weight_decay=0.01,                  # Weight decay
    save_steps=10,                      # Save checkpoint every 10 steps
    logging_dir="logs",                 # Log directory
    logging_steps=10,                   # Log every 10 steps
    fp16=True,                          # Enable mixed precision (if supported)
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()


In Algeria, East, Center the most eaten food is nan
In Algeria, Miliana the most eaten food is taam-haalhaal
In Algeria, Center, Algiers the most eaten food is khdawej ala derbuz
In Algeria, East, Constantine the most eaten food is zaligu
In Algeria, Constantine, East the most eaten food is shakhshoukhet dhfer
In Algeria, Algiers the most eaten food is shorba beyda
In Algeria, Center the most eaten food is Dolma qarnun
In Algeria, Ouargla, South the most eaten food is Mardoud
In Algeria, Constantine the most eaten food is shebah sefra/ garnisher of the meal
In Algeria, Touareg , South the most eaten food is Legmi/ Palm drink
In Algeria, Algiers, Center the most eaten food is the baby in his mother's lap
In Algeria, Biskra the most eaten food is dobara
In Algeria, Bousaada the most eaten food is algerian chili dish
In Algeria, Ouest the most eaten food is garantita, chickpea gello
In Algeria, Oran, Ouest the most eaten food is hrira, algerian hrira
In Algeria, Algiers, central Algeria t

In [11]:
trainer.save_model()  # Saves the model (PyTorch model weights)
tokenizer.save_pretrained(training_args.output_dir)  # Save the tokenizer

('trainer\\tokenizer_config.json',
 'trainer\\special_tokens_map.json',
 'trainer\\vocab.json',
 'trainer\\merges.txt',
 'trainer\\added_tokens.json',
 'trainer\\tokenizer.json')

In [13]:
model = AutoModelForCausalLM.from_pretrained("trainer")

In [14]:
# Fine-tuned model

classifier = pipeline(
    "text-generation", 
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    max_length=50
)